In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
print pd.__version__

0.16.1


In [2]:
ratingData = pd.read_csv("../weekend1/movieData/ratings.dat",sep = "::",names = ['UserID','MovieID','Rating','Timestamp'])
movieData = pd.read_table("../weekend1/movieData/movies.dat",sep="::", names = ["MovieID","Title","Genres"])
userData = pd.read_table("../weekend1/movieData/users.dat", sep="::", names = ["UserID","Gender","Age","Occupation","Zip-code"])

/usr/local/lib/python2.7/site-packages/pandas/io/parsers.py:648: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
  ParserWarning)


In [3]:
ratingData.Timestamp = pd.to_datetime(ratingData.Timestamp, unit="s")
movieData = pd.concat([movieData,movieData.Genres.str.get_dummies(sep = "|")],axis=1)
data = userData.merge(ratingData.merge(movieData))

In [4]:
mostReviewedMoviesData = data.groupby("Title").filter(lambda x: x.shape[0]>=100) #keep only those movies that have been reviewed at least 100 times
statsByGender = mostReviewedMoviesData.pivot_table("Rating", index="Title",columns="Gender",aggfunc = [np.mean,np.std]) #get per-gender avg, std of ratings per movie

In [5]:
statsByGender["meanDifference"] = statsByGender["mean"]["F"] - statsByGender["mean"]["M"] # get diff in mean rating between genders
statsByGender.sort("meanDifference", ascending = False, inplace=True)
print "Movies women tended to like more than men: \n", statsByGender.head(), "\n"
print "Movies men tended to like more than women: \n", statsByGender[::-1].head(), "\n"
statsByGender.sort(("std","F"), ascending = False, inplace=True)
print "Movies women tended to disagree on: \n", statsByGender.head(), "\n"
print "Movies women tended to agree on: \n", statsByGender[::-1].head(), "\n"

Movies women tended to like more than men: 
                             mean                 std           meanDifference
Gender                          F         M         F         M               
Title                                                                         
Pet Sematary II (1992)   2.833333  1.858696  1.340560  0.978691       0.974638
Cutthroat Island (1995)  3.200000  2.341270  1.361114  1.160432       0.858730
Dirty Dancing (1987)     3.790378  2.959596  1.105181  1.087738       0.830782
Air Bud (1997)           3.057143  2.233766  1.211291  1.086957       0.823377
Home Alone 3 (1997)      2.486486  1.683761  1.556735  0.934488       0.802726 

Movies men tended to like more than women: 
                                                    mean                 std  \
Gender                                                 F         M         F   
Title                                                                          
Friday the 13th Part V: A New Beginnin

In [6]:
data = data.assign(high_rating = lambda x: x.Rating>=4)

In [7]:
print data.query("high_rating")
data.query("high_rating").groupby("Title").size()

         UserID Gender  Age  Occupation Zip-code  MovieID  Rating  \
0             1      F    1          10    48067     1193       5   
3             1      F    1          10    48067     3408       4   
4             1      F    1          10    48067     2355       5   
6             1      F    1          10    48067     1287       5   
7             1      F    1          10    48067     2804       5   
8             1      F    1          10    48067      594       4   
9             1      F    1          10    48067      919       4   
10            1      F    1          10    48067      595       5   
11            1      F    1          10    48067      938       4   
12            1      F    1          10    48067     2398       4   
13            1      F    1          10    48067     2918       4   
14            1      F    1          10    48067     1035       5   
15            1      F    1          10    48067     2791       4   
17            1      F    1       

Title
$1,000,000 Duck (1971)                              11
'Night Mother (1986)                                31
'Til There Was You (1997)                           12
'burbs, The (1989)                                  91
...And Justice for All (1979)                      120
10 Things I Hate About You (1999)                  353
101 Dalmatians (1961)                              329
101 Dalmatians (1996)                              130
12 Angry Men (1957)                                525
13th Warrior, The (1999)                           335
187 (1997)                                          10
2 Days in the Valley (1996)                        128
20 Dates (1998)                                     45
20,000 Leagues Under the Sea (1954)                343
200 Cigarettes (1999)                               55
2001: A Space Odyssey (1968)                      1270
2010 (1984)                                        231
24 7: Twenty Four Seven (1997)                       3
28 D

In [14]:
def split_train_test(df,sample=0.4, testSetColumnName="testSet"):
    if np.random.random() < sample:
        df.ix[:, testSetColumnName] = True
    return df

In [26]:
data["testSet"] = False
data2 = data.groupby("UserID").apply(split_train_test)

In [31]:
data2.head(100)

,UserID,Gender,Age,Occupation,Zip-code,MovieID,Rating,Timestamp,Title,Genres,...,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,high_rating,testSet
0,1,F,1,10,48067,1193,5,2000-12-31 22:12:40,One Flew Over the Cuckoo's Nest (1975),Drama,...,0,0,0,0,0,0,0,0,True,False
1,1,F,1,10,48067,661,3,2000-12-31 22:35:09,James and the Giant Peach (1996),Animation|Children's|Musical,...,0,1,0,0,0,0,0,0,False,False
2,1,F,1,10,48067,914,3,2000-12-31 22:32:48,My Fair Lady (1964),Musical|Romance,...,0,1,0,1,0,0,0,0,False,False
3,1,F,1,10,48067,3408,4,2000-12-31 22:04:35,Erin Brockovich (2000),Drama,...,0,0,0,0,0,0,0,0,True,False
4,1,F,1,10,48067,2355,5,2001-01-06 23:38:11,"Bug's Life, A (1998)",Animation|Children's|Comedy,...,0,0,0,0,0,0,0,0,True,False
5,1,F,1,10,48067,1197,3,2000-12-31 22:37:48,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance,...,0,0,0,1,0,0,0,0,False,False
6,1,F,1,10,48067,1287,5,2000-12-31 22:33:59,Ben-Hur (1959),Action|Adventure|Drama,...,0,0,0,0,0,0,0,0,True,False
7,1,F,1,10,48067,2804,5,2000-12-31 22:11:59,"Christmas Story, A (1983)",Comedy|Drama,...,0,0,0,0,0,0,0,0,True,False
8,1,F,1,10,48067,594,4,2000-12-31 22:37:48,Snow White and the Seven Dwarfs (1937),Animation|Children's|Musical,...,0,1,0,0,0,0,0,0,True,False
9,1,F,1,10,48067,919,4,2000-12-31 22:22:48,"Wizard of Oz, The (1939)",Adventure|Children's|Drama|Musical,...,0,1,0,0,0,0,0,0,True,False
